In [1]:
#@title # Устанавливаем библиотеки, загружаем рабочие папки, вводим токен ngrok, запускаем приложение на сервере
!pip install fastapi uvicorn nest-asyncio pyngrok jinja2 ultralytics matplotlib
!pip install ffmpeg-python opencv-python-headless

from fastapi import FastAPI, Request, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import asyncio
import getpass
import os
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import gdown
from IPython.display import clear_output
import cv2
import ffmpeg
from fastapi import Form
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import uuid
clear_output()

# URL файла на Google Диске
url = 'https://drive.google.com/uc?id=1TXDIQukbe8VLqFfvTIO8vqT0Xcs9qwv5'

# Путь, куда будет сохранен файл
output = '/content/Logo.png'

# Загрузка файла
gdown.download(url, output, quiet=False)

# Перемещение файла в нужную директорию (если необходимо)
shutil.move(output, '/content/Logo.png')

# Введите ваш ngrok authtoken один раз
if "NGROK_AUTH_TOKEN" not in os.environ:
    ngrok_auth_token = getpass.getpass("Введите ваш ngrok authtoken: ")
    os.environ["NGROK_AUTH_TOKEN"] = ngrok_auth_token

# Установим токен из переменной окружения
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

app = FastAPI()

# Создание необходимых директорий
os.makedirs("/content/data/static", exist_ok=True)
os.makedirs("/content/data/templates", exist_ok=True)

# Переместите файл изображения в директорию статических файлов
shutil.copy("/content/Logo.png", "/content/data/static/Logo.png")

style = """
body {
    background-color: #F0F0F0;
    font-family: Arial, sans-serif;
    color: #333;
    margin: 0;
    height: 100vh;
    display: grid;
    grid-template-rows: 250px 1fr;  /* Установка высоты верхнего блока */
    grid-template-columns: 200px 1fr;
}
.logo {
    grid-row: 1 / 2;
    grid-column: 1 / 3;
    display: flex;
    align-items: center;
    justify-content: center;
    background-color: #D0D0D0;
    padding: 20px;
    border-bottom: 2px solid #FFFFFF; /* Тонкая белая линия */
}
.logo img {
    max-width: 100%;
    max-height: 100%;
    object-fit: contain;  /* Сохраняет пропорции изображения */
}
.sidebar {
    grid-row: 2 / 3;
    grid-column: 1 / 2;
    background-color: #E0E0E0;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: flex-start;
    padding: 20px;
    border-right: 2px solid #A0A0A0;
}
.sidebar h1 {
    color: #000000;
    margin-bottom: 20px;
}
.sidebar .button {
    width: 100%;
    margin: 10px 0;
    padding: 15px;
    font-size: 18px;
    background-color: #A0A0A0;
    color: #000000;
    border: none;
    border-radius: 12px;
    text-align: center;
    cursor: pointer;
}
.sidebar .button:hover {
    background-color: #C0C0C0;
}
.main-content {
    grid-row: 2 / 3;
    grid-column: 2 / 3;
    display: flex;
    flex-direction: column;
    align-items: flex-start;
    justify-content: flex-start;
    text-align: left;
    padding: 20px;
    background-color: #E0E0E0;
    border-top: 0px solid #FFFFFF; /* Тонкая белая линия между верхним блоком и основным контентом */
}
.main-content h1 {
    font-size: 28px;
    margin-bottom: 20px;
    color: #000000;
}
.main-content p {
    font-size: 18px;
    margin-bottom: 10px;
    color: #333333;
}
.main-content img, .main-content video {
    max-width: 100%;
    height: auto;
    margin: 20px 0;
}
"""

index_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
    </div>
    <div class="main-content" style="text-align: left;">
        <h1>Марафон 16.07.2024 Веб-Приложение</h1>
        <p>Университет искусственного интеллекта</p>
    </div>
</body>
</html>
"""

image_html = """
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" type="text/css" href="/static/style.css">
    <style>{style}</style>
</head>
<body>
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
    </div>
    <div class="main-content">
        <h1>Распознавание изображений</h1>
        <p>Загрузите изображение для распознавания объектов:</p>

        <!-- Форма для загрузки изображения и выбора класса -->
        <form action="/upload-image/" enctype="multipart/form-data" method="post">
            <input type="file" name="file" accept="image/*">
            <label for="class_select">Выберите класс для распознавания:</label>
            <select name="class_select" id="class_select">
                <option value="0" {% if selected_class == 0 %}selected{% endif %}>Человек</option>
                <option value="1" {% if selected_class == 1 %}selected{% endif %}>Велосипед</option>
                <option value="2" {% if selected_class == 2 %}selected{% endif %}>Машина</option>
            </select>
            <input type="submit" value="Загрузить" class="button">
        </form>

        <!-- Отображение результата -->
        {% if output_image_path %}
            <h2>Результат:</h2>
            <img src="{{ output_image_path }}" alt="Распознанное изображение">
        {% endif %}
        {% if error %}
            <h2>{{ error }}</h2>
        {% endif %}
    </div>
</body>
</html>
"""

video_html = """
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" type="text/css" href="/static/style.css">
    <style>{style}</style>
</head>
<body>
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
    </div>
    <div class="main-content">
        <h1>Распознавание видео</h1>
        <p>Загрузите видео для распознавания объектов:</p>

        <!-- Форма для загрузки видео и выбора класса -->
        <form action="/upload-video/" enctype="multipart/form-data" method="post">
            <input type="file" name="file" accept="video/*">
            <label for="class_select">Выберите класс для распознавания:</label>
            <select name="class_select" id="class_select">
                <option value="0" {% if selected_class == 0 %}selected{% endif %}>Человек</option>
                <option value="2" {% if selected_class == 2 %}selected{% endif %}>Машина</option>
                <option value="41" {% if selected_class == 41 %}selected{% endif %}>Стаканчик</option>
            </select>
            <input type="submit" value="Загрузить" class="button">
        </form>

        <!-- Отображение результата -->
        {% if output_video_url %}
            <h2>Результат:</h2>
            <video width="800" height="600" controls>
                <source src="{{ output_video_url }}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
        {% endif %}
        {% if error %}
            <h2>{{ error }}</h2>
        {% endif %}
    </div>
</body>
</html>
"""


interface_html = """
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" type="text/css" href="/static/style.css">
    <style>{style}</style>
</head>
<body>
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
    </div>
    <div class="main-content">
    </div>
</body>
</html>
"""

# Сохранение файлов шаблонов
with open("/content/data/templates/index.html", "w") as f:
    f.write(index_html.format(style=style))

with open("/content/data/templates/image.html", "w") as f:
    f.write(image_html)

with open("/content/data/templates/video.html", "w") as f:
    f.write(video_html)

with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

with open("/content/data/static/style.css", "w") as f:
    f.write(style)

app.mount("/static", StaticFiles(directory="/content/data/static"), name="static")
templates = Jinja2Templates(directory="/content/data/templates")

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/image", response_class=HTMLResponse)
async def image_page(request: Request):
    return templates.TemplateResponse("image.html", {"request": request})

@app.get("/video", response_class=HTMLResponse)
async def video_page(request: Request):
    return templates.TemplateResponse("video.html", {"request": request})

@app.get("/interface", response_class=HTMLResponse)
async def interface_page(request: Request):
    return templates.TemplateResponse("interface.html", {"request": request})

@app.post("/upload-image/", response_class=HTMLResponse)
async def upload_image(request: Request, file: UploadFile = File(...), class_select: int = Form(...)):
    try:
        img_path = f"/content/data/static/{file.filename}"
        with open(img_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Обработка изображения с помощью модели YOLO и выбранного класса
        results = model(img_path, classes=class_select)

        # Визуализация и сохранение результатов
        if len(results) > 0 and results[0].boxes:
            result_image = results[0].plot()  # Визуализация результата

            if result_image is not None:
                # Сохраняем изображение с боксами
                output_image_path = f"/content/data/static/detected_{file.filename}"

                # Использование PIL для сохранения изображения
                result_image_pil = Image.fromarray(result_image)
                result_image_pil.save(output_image_path)

                return templates.TemplateResponse("image.html", {"request": request, "output_image_path": f"/static/detected_{file.filename}", "selected_class": class_select})
            else:
                return templates.TemplateResponse("image.html", {"request": request, "error": "Ошибка при обработке изображения", "selected_class": class_select})
        else:
            return templates.TemplateResponse("image.html", {"request": request, "error": "Объекты не найдены", "selected_class": class_select})

    except Exception as e:
        return templates.TemplateResponse("image.html", {"request": request, "error": "Internal Server Error", "selected_class": class_select})


@app.post("/upload-video/", response_class=HTMLResponse)
async def upload_video(request: Request, file: UploadFile = File(...), class_select: int = Form(...)):
    try:
        unique_id = str(uuid.uuid4())
        video_filename = f"{unique_id}_{file.filename}"
        output_filename = f"{unique_id}_detected_{file.filename}"
        h264_output_filename = f"{unique_id}_detected_{file.filename.replace('.mp4', '_h264.mp4')}"

        video_path = f"/content/data/static/{video_filename}"
        output_path = f"/content/data/static/{output_filename}"
        h264_output_path = f"/content/data/static/{h264_output_filename}"

        # Сохранение загруженного видео файла
        with open(video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Обработка видео с помощью модели YOLO и выбранного класса
        model = YOLO('yolov8n.pt')
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception("Ошибка открытия видеофайла")

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        object_found = False
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            results = model(frame, classes=class_select)
            if results[0].boxes:
                object_found = True
            result_frame = results[0].plot()
            out.write(cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR))  # Сохраняем кадры в видеофайл
            frame_count += 1

        cap.release()
        out.release()

        if object_found:
            # Конвертируем видео в кодек H.264 с помощью ffmpeg
            ffmpeg.input(output_path).output(h264_output_path, vcodec="libx264").run()

            # Используем прямой путь к видео
            output_video_url = f"/static/{h264_output_filename}"

            # Удаление временных файлов
            os.remove(video_path)
            os.remove(output_path)

            return templates.TemplateResponse("video.html", {"request": request, "output_video_url": output_video_url, "selected_class": class_select})
        else:
            os.remove(video_path)
            return templates.TemplateResponse("video.html", {"request": request, "error": "Объекты класса не найдены", "selected_class": class_select})

    except Exception as e:
        print(f"Ошибка: {e}")  # Логируем ошибку для отладки
        if os.path.exists(video_path):
            os.remove(video_path)
        if os.path.exists(output_path):
            os.remove(output_path)
        return templates.TemplateResponse("video.html", {"request": request, "error": "Internal Server Error", "selected_class": class_select})
    except Exception as e:
        print(f"Ошибка: {e}")  # Логируем ошибку для отладки
        if os.path.exists(video_path):
            os.remove(video_path)
        if os.path.exists(output_path):
            os.remove(output_path)
        return templates.TemplateResponse("video.html", {"request": request, "error": "Internal Server Error", "selected_class": class_select})


# Инициализация модели YOLOv8
model = YOLO('yolov8n.pt')  # Замените на путь к вашей модели, если необходимо

# Закрываем все активные туннели
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

# Туннелирование с помощью ngrok
public_url = ngrok.connect(8000)
print(f" * Tunnel URL: {public_url}")

# Запуск приложения на локальном порту
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

async def start_server():
    await server.serve()

server_task = asyncio.ensure_future(start_server())

   ---------------------------------------- 0.0/876.2 kB ? eta -:--:--
   --------------------------------------- 876.2/876.2 kB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 7.8/7.8 MB 44.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 62.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 11.5/11.5 MB 89.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 51.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   --------- ------------------------------ 10.7/44.5 MB 51.7 MB/s eta 0:00:01
   ------------- -------------------------- 15.5/44.5 MB 37.4 MB/s eta 0:00:01
   --------------- ---------------

ModuleNotFoundError: No module named 'gdown'

## **1. Работа с Веб-интерфейсом**

### **Шаг 1: Добавление заголовка**

**Основные HTML-теги и их описание:**

- `<!DOCTYPE html>` - Объявление типа документа. Указывает браузеру, что документ написан на HTML5.

- `<html>` - Корневой элемент HTML-документа.

- `<head>` - Раздел документа, содержащий метаинформацию, стили и скрипты:
  - `<title>` - Заголовок документа, который отображается на вкладке браузера.
  - `<meta>` - Метаинформация о документе, например, кодировка.
  - `<link>` - Включение внешних ресурсов, таких как стили CSS.
  - `<style>` - Встроенные стили CSS.
  - `<script>` - Встроенные или внешние скрипты JavaScript.
  - `<h1>` до `<h6>` - Заголовки разного уровня.

Добавим заголовок на страницу интерфейса. Мы вставим элемент `<h1>` в блок `main-content`.


In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в верхней части страницы -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
    </div>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 2: Добавление текста**



**Теги для работы с текстом:**

- `<body>` - Основное содержимое страницы:
  - `<div>` - Блочный элемент, используется для группировки содержимого.
  - `<p>` - Абзац текста.
  - `<a>` - Гиперссылка.
  - `<button>` - Кнопка.
  - `<form>` - Форма для ввода данных.
  - `<input>` - Поле ввода данных в форме.
  - `<label>` - Надпись для элемента формы.
  - `<br>` - Перенос строки.
  - `<span>` - Встроенный элемент, используется для стилизации части текста.


Добавим абзац с текстом. Мы вставим элемент `<p>` под заголовком `<h1>` в блок `main-content`.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в верхней части страницы -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим основные методы редактирования дизайна приложения.</p> <!-- Добавляем текст здесь -->
    </div>
</body>
</html>
"""

with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 3: Добавление изображения**

**Теги для работы с изображениями:**

- `<img>` - Тег для встраивания изображения.
  - `src` - Путь к изображению.
  - `alt` - Альтернативный текст.
  - `width` и `height` - Атрибуты для указания размеров изображения.
  - `style` - Встроенные стили для изображения.

Вставим изображение на страницу. Мы вставим элемент `<img>` под абзацем `<p>` в блок `main-content`.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в верхней части страницы -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
    </div>
</body>
</html>
"""

with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 4: Добавление таблицы**

**Теги для работы с таблицами:**

- `<table>` - Элемент для создания таблицы.
  - `<tr>` - Строка таблицы.
  - `<th>` - Заголовочная ячейка таблицы.
  - `<td>` - Обычная ячейка таблицы.

Создадим таблицу и заполним её данными. Мы вставим элемент `<table>` под изображением в блок `main-content`.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в верхней части страницы -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
    </div>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 5: Добавление списка**

  **Теги для работы со списками:**
  - `<ul>` и `<ol>` - Нумерованные и ненумерованные списки.
  - `<li>` - Элемент списка.

Создадим нумерованный список. Мы вставим элемент `<ol>` под таблицей в блок `main-content`.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в верхней части страницы -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
        <!-- Добавляем нумерованный список здесь -->
        <ol style="margin-top: 20px;">
            <li>Элемент 1</li>
            <li>Элемент 2</li>
            <li>Элемент 3</li>
        </ol>
    </div>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 6: Изменение фона**

**Основные CSS-команды и их краткое описание**

- `body` - Элемент для стилизации всего содержимого страницы.
  - `background-color` - Устанавливает цвет фона.
  - `font-family` - Задает семейство шрифтов для текста.
  - `color` - Задает цвет текста.
  - `margin` - Устанавливает отступы вокруг всего содержимого.
  - `height` - Устанавливает высоту элемента.
  - `display` - Задает тип отображения элемента (например, `grid`).
  - `grid-template-rows` - Определяет количество строк и их размер в сетке.
  - `grid-template-columns` - Определяет количество колонок и их размер в сетке.

- `display` - Свойство для определения способа отображения элемента.
  - `flex` - Определяет гибкий контейнер.
  - `grid` - Определяет контейнер сетки.

- `grid-row` - Определяет, сколько строк занимает элемент в сетке и его начальную и конечную позицию.
- `grid-column` - Определяет, сколько колонок занимает элемент в сетке и его начальную и конечную позицию.

- `align-items` - Выравнивает элементы по вертикали внутри flex или grid контейнера.
- `justify-content` - Выравнивает элементы по горизонтали внутри flex или grid контейнера.

- `padding` - Устанавливает внутренние отступы элемента.

- `border` - Устанавливает границу элемента.
  - `border-bottom` - Устанавливает нижнюю границу.
  - `border-right` - Устанавливает правую границу.
  - `border-radius` - Задает радиус скругления углов границы.

- `width` - Устанавливает ширину элемента.
- `height` - Устанавливает высоту элемента.

- `background-color` - Устанавливает цвет фона элемента.

- `font-size` - Задает размер шрифта текста.
- `color` - Задает цвет текста.

- `margin` - Устанавливает внешние отступы элемента.
  - `margin-bottom` - Устанавливает нижний внешний отступ элемента.

- `text-align` - Выравнивает текст внутри элемента.

- `cursor` - Определяет вид курсора при наведении на элемент.

- `overflow-y` - Определяет поведение переполнения содержимого по вертикали (например, прокрутка).



Изменим цвет фона главного участка страницы. Мы обновим файл стилей `style.css`.

In [ ]:
#@title Обновление стиля страницы
style = """
body {
    background-color: #F0F0F0; /* Фон для всего тела документа */
    font-family: Arial, sans-serif; /* Шрифт для текста */
    color: #333; /* Цвет текста */
    margin: 0; /* Отступы вокруг тела документа */
    height: 100vh; /* Высота окна браузера */
    display: grid; /* Использование CSS Grid Layout */
    grid-template-rows: 250px 1fr;  /* Установка высоты верхнего блока */
    grid-template-columns: 200px 1fr; /* Ширина боковой панели и основной ячейки */
}
.logo {
    grid-row: 1 / 2; /* Позиционирование логотипа по сетке */
    grid-column: 1 / 3; /* Логотип занимает две колонки */
    display: flex; /* Использование Flexbox для выравнивания содержимого */
    align-items: center; /* Выравнивание по вертикали по центру */
    justify-content: center; /* Выравнивание по горизонтали по центру */
    background-color: #D0D0D0; /* Фон для логотипа */
    padding: 20px; /* Внутренние отступы */
    border-bottom: 2px solid #FFFFFF; /* Тонкая белая линия снизу */
}
.logo img {
    max-width: 100%; /* Максимальная ширина изображения */
    max-height: 100%; /* Максимальная высота изображения */
    object-fit: contain;  /* Сохраняет пропорции изображения */
}
.sidebar {
    grid-row: 2 / 3; /* Позиционирование боковой панели по сетке */
    grid-column: 1 / 2; /* Боковая панель занимает одну колонку */
    background-color: #E0E0E0; /* Фон для боковой панели */
    display: flex; /* Использование Flexbox для выравнивания содержимого */
    flex-direction: column; /* Направление Flexbox - колонка */
    align-items: center; /* Выравнивание по центру по горизонтали */
    justify-content: flex-start; /* Выравнивание по вертикали к началу */
    padding: 20px; /* Внутренние отступы */
    border-right: 2px solid #A0A0A0; /* Тонкая серая линия справа */
}
.sidebar h1 {
    color: #000000; /* Цвет заголовка */
    margin-bottom: 20px; /* Нижний отступ */
}
.sidebar .button {
    width: 100%; /* Ширина кнопки */
    margin: 10px 0; /* Отступы сверху и снизу */
    padding: 15px; /* Внутренние отступы */
    font-size: 18px; /* Размер шрифта */
    background-color: #A0A0A0; /* Фон кнопки */
    color: #000000; /* Цвет текста кнопки */
    border: none; /* Без рамки */
    border-radius: 12px; /* Скругленные углы */
    text-align: center; /* Выравнивание текста по центру */
    cursor: pointer; /* Курсор при наведении */
}
.sidebar .button:hover {
    background-color: #C0C0C0; /* Фон кнопки при наведении */
}
.main-content {
    grid-row: 2 / 3; /* Позиционирование основного контента по сетке */
    grid-column: 2 / 3; /* Основной контент занимает одну колонку */
    display: flex; /* Использование Flexbox для выравнивания содержимого */
    flex-direction: column; /* Направление Flexbox - колонка */
    align-items: flex-start; /* Выравнивание по началу по горизонтали */
    justify-content: flex-start; /* Выравнивание по началу по вертикали */
    text-align: left; /* Текст выравнивается по левому краю */
    padding: 20px; /* Внутренние отступы */
    background-color: #FFFFFF; /* Фон основной ячейки - белый */
    border-top: 2px solid #FFFFFF; /* Тонкая белая линия между верхним блоком и основным контентом */
}
.main-content h1 {
    font-size: 28px; /* Размер шрифта заголовка */
    margin-bottom: 20px; /* Нижний отступ заголовка */
    color: #000000; /* Цвет заголовка */
}
.main-content p {
    font-size: 18px; /* Размер шрифта параграфа */
    margin-bottom: 10px; /* Нижний отступ параграфа */
    color: #333333; /* Цвет текста параграфа */
}
.main-content img, .main-content video {
    max-width: 100%; /* Максимальная ширина изображения или видео */
    height: auto; /* Автоматическая высота для сохранения пропорций */
    margin: 20px 0; /* Отступы сверху и снизу */
}
"""

# Сохраняем изменения в файл стилей
with open("/content/data/static/style.css", "w") as f:
    f.write(style)

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

    background-color: #FFFFFF; /* Белый фон для основного контента */

### **Шаг 7: Добавление кнопки**

**Основные HTML-теги для работы с кнопками**

- `<button>` - Создает интерактивную кнопку.
  - `type` - Определяет тип кнопки (например, "button", "submit", "reset").
  - `onclick` - Обработчик событий, который срабатывает при щелчке на кнопке.


Добавим кнопку в основное окно.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в левом верхнем углу -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
        <!-- Добавляем нумерованный список здесь -->
        <ol style="margin-top: 20px;">
            <li>Элемент 1</li>
            <li>Элемент 2</li>
            <li>Элемент 3</li>
        </ol>
        <!-- Добавляем кнопку здесь -->
        <button class="button" onclick="alert('Кнопка нажата!')">Нажми меня</button>
    </div>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 8: Добавление формы ввода**

**Основные HTML-теги для работы с кнопками и формами**

- `<form>` - Создает форму для ввода данных.
  - `style` - Встроенные стили CSS для элемента.

- `<label>` - Создает подпись для элемента формы.
  - `for` - Связывает подпись с элементом формы по идентификатору (id).

- `<input>` - Создает интерактивный элемент формы.
  - `type` - Определяет тип элемента формы (например, "text", "submit").
  - `id` - Уникальный идентификатор элемента формы.
  - `name` - Имя элемента формы.
  - `value` - Значение элемента формы.

Добавим форму для ввода информации с одной ячейкой ввода на страницу.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип в левом верхнем углу -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
        <!-- Добавляем нумерованный список здесь -->
        <ol style="margin-top: 20px;">
            <li>Элемент 1</li>
            <li>Элемент 2</li>
            <li>Элемент 3</li>
        </ol>
        <!-- Добавляем кнопку здесь -->
        <button class="button" onclick="alert('Кнопка нажата!')">Нажми меня</button>
        <!-- Добавляем форму ввода здесь -->
        <form style="margin-top: 20px;">
            <label for="inputField">Введите текст:</label>
            <input type="text" id="inputField" name="inputField">
            <input type="submit" value="Отправить">
        </form>
    </div>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 9: Отображение введенного значения**

**Основные команды JavaScript**

- `<script>` - Тег для включения и выполнения JavaScript-кода.
- `function` - Ключевое слово для объявления функции.
- `var` - Ключевое слово для объявления переменной.
- `document.getElementById` - Метод для получения элемента по его `id`.
- `value` - Свойство для получения или установки значения элемента формы.
- `innerText` - Свойство для установки или получения текста внутри элемента.


Добавим функциональность для формы, чтобы при нажатии кнопки отображалось введенное значение на экране.

In [ ]:
#@title Страница Интерфейс
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип сверху -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
        <!-- Добавляем нумерованный список здесь -->
        <ol style="margin-top: 20px;">
            <li>Элемент 1</li>
            <li>Элемент 2</li>
            <li>Элемент 3</li>
        </ol>
        <!-- Добавляем кнопку здесь -->
        <button class="button" onclick="alert('Кнопка нажата!')">Нажми меня</button>
        <!-- Добавляем форму ввода здесь -->
        <form id="inputForm" style="margin-top: 20px;">
            <label for="inputField">Введите текст:</label>
            <input type="text" id="inputField" name="inputField">
            <input type="button" value="Отправить" onclick="displayInput()">
        </form>
        <!-- Отображение введенного текста -->
        <p id="outputText"></p>
    </div>

    <script>
        // Функция для отображения введенного текста
        function displayInput() {{
            var input = document.getElementById('inputField').value; // Получаем значение из поля ввода
            document.getElementById('outputText').innerText = 'Вы ввели: ' + input; // Отображаем введенное значение
        }}
    </script>
</body>
</html>
"""

# Сохраняем изменения в шаблон
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

### **Шаг 10: Создание дополнительной кнопки меню и новой страницы (будет видно только на странице интерфейс)**

1. В шаблоне интерфейсной страницы добавляем новую кнопку в боковое меню, которая будет вести на новую страницу "Тест".

2. Создаем новый шаблон для страницы "Тест", который будет отображаться при переходе по соответствующей кнопке меню.

3. Обновляем обработчик в FastAPI для обработки запроса к новой странице.


In [ ]:
#@title Обновление шаблона "Интерфейс" для добавления кнопки "ТЕСТ"
interface_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип сверху -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
        <button class="button" onclick="window.location.href='/test'">ТЕСТ</button> <!-- Добавляем кнопку "ТЕСТ" -->
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Интерфейс</h1> <!-- Добавляем заголовок здесь -->
        <p>Здесь мы рассмотрим создание кнопок, изменение цвета и вида окна.</p> <!-- Добавляем текст здесь -->
        <img src="/static/Logo.png" alt="Пример изображения" style="width:200px;"> <!-- Добавляем изображение здесь -->
        <!-- Добавляем таблицу здесь -->
        <table border="1" style="margin-top: 20px;">
            <tr>
                <th>Заголовок 1</th>
                <th>Заголовок 2</th>
                <th>Заголовок 3</th>
            </tr>
            <tr>
                <td>Ячейка 1</td>
                <td>Ячейка 2</td>
                <td>Ячейка 3</td>
            </tr>
            <tr>
                <td>Ячейка 4</td>
                <td>Ячейка 5</td>
                <td>Ячейка 6</td>
            </tr>
        </table>
        <!-- Добавляем нумерованный список здесь -->
        <ol style="margin-top: 20px;">
            <li>Элемент 1</li>
            <li>Элемент 2</li>
            <li>Элемент 3</li>
        </ol>
        <!-- Добавляем кнопку здесь -->
        <button class="button" onclick="alert('Кнопка нажата!')">Нажми меня</button>
        <!-- Добавляем форму ввода здесь -->
        <form id="inputForm" style="margin-top: 20px;">
            <label for="inputField">Введите текст:</label>
            <input type="text" id="inputField" name="inputField">
            <input type="button" value="Отправить" onclick="displayInput()">
        </form>
        <!-- Отображение введенного текста -->
        <p id="outputText"></p>
    </div>

    <script>
        // Функция для отображения введенного текста
        function displayInput() {{
            var input = document.getElementById('inputField').value; // Получаем значение из поля ввода
            document.getElementById('outputText').innerText = 'Вы ввели: ' + input; // Отображаем введенное значение
        }}
    </script>
</body>
</html>
"""

# Сохраняем изменения в шаблон "Интерфейс"
with open("/content/data/templates/interface.html", "w") as f:
    f.write(interface_html.format(style=style))

In [ ]:
#@title Создаем шаблон Шаблон для новой страницы "Тест"
test_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
        <button class="button" onclick="window.location.href='/test'">ТЕСТ</button>
    </div>
    <div class="main-content">
    </div>
    <div class="right-sidebar">
    </div>
</body>
</html>
"""

# Сохраняем шаблон
with open("/content/data/templates/test.html", "w") as f:
    f.write(test_html.format(style=style))

In [ ]:
#@title Создаем обработчик для новой страницы "Тест"
@app.get("/test", response_class=HTMLResponse)
async def get_test(request: Request):
    return templates.TemplateResponse("test.html", {"request": request})

### **Шаг 11: Создание функциональной страницы для математических операций на новой странице**


Добавим форму для ввода двух чисел и кнопки для выполнения операций сложения, вычитания, умножения и деления. Результат будет отображаться на экране.

In [ ]:
#@title Страница для математических операций
test_html = """
<!DOCTYPE html>
<html>
<head>
    <style>{style}</style>
</head>
<body>
    <!-- Логотип сверху -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo">
    </div>
    <!-- Боковое меню с кнопками для навигации -->
    <div class="sidebar">
        <h1>Меню</h1>
        <button class="button" onclick="window.location.href='/'">Главная</button>
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button>
        <button class="button" onclick="window.location.href='/image'">Изображение</button>
        <button class="button" onclick="window.location.href='/video'">Видео</button>
        <button class="button" onclick="window.location.href='/monitoring'">Мониторинг</button>
        <button class="button" onclick="window.location.href='/test'">Тест</button>
    </div>
    <!-- Основной контент страницы -->
    <div class="main-content">
        <h1>Математические операции</h1> <!-- Добавляем заголовок здесь -->
        <p>Введите два числа и выберите операцию:</p> <!-- Добавляем текст здесь -->

        <!-- Добавляем форму для ввода чисел -->
        <form id="mathForm" style="margin-top: 20px;">
            <label for="num1">Число 1:</label>
            <input type="number" id="num1" name="num1">
            <br><br>
            <label for="num2">Число 2:</label>
            <input type="number" id="num2" name="num2">
            <br><br>
            <input type="button" value="Сложить" onclick="performOperation('add')" class="button">
            <input type="button" value="Вычесть" onclick="performOperation('subtract')" class="button">
            <input type="button" value="Умножить" onclick="performOperation('multiply')" class="button">
            <input type="button" value="Делить" onclick="performOperation('divide')" class="button">
        </form>
        <!-- Отображение результата -->
        <p id="result"></p>
    </div>

    <script>
        // Функция для выполнения математической операции
        function performOperation(operation) {{
            var num1 = parseFloat(document.getElementById('num1').value);
            var num2 = parseFloat(document.getElementById('num2').value);
            var result;

            if (isNaN(num1) || isNaN(num2)) {{
                result = 'Пожалуйста, введите оба числа.';
            }} else {{
                if (operation === 'add') {{
                    result = num1 + num2;
                }} else if (operation === 'subtract') {{
                    result = num1 - num2;
                }} else if (operation === 'multiply') {{
                    result = num1 * num2;
                }} else if (operation === 'divide') {{
                    if (num2 !== 0) {{
                        result = num1 / num2;
                    }} else {{
                        result = 'Ошибка: деление на ноль.';
                    }}
                }}
            }}

            document.getElementById('result').innerText = 'Результат: ' + result;
        }}
    </script>
</body>
</html>
"""

# Сохраняем новый шаблон
with open("/content/data/templates/test.html", "w") as f:
    f.write(test_html.format(style=style))

## **2. Распознование объектов на изображениях**

In [ ]:
#@title Функция для загрузки и обработки изображения с помощью модели YOLO
@app.post("/upload-image/", response_class=HTMLResponse)
async def upload_image(request: Request, file: UploadFile = File(...), class_select: int = Form(...)):
    try:
        # Сохранение загруженного изображения на диск
        img_path = f"/content/data/static/{file.filename}"
        with open(img_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Обработка изображения с помощью модели YOLO и выбранного класса
        results = model(img_path, classes=class_select)

        # Визуализация и сохранение результатов
        if len(results) > 0 and results[0].boxes:
            result_image = results[0].plot()  # Визуализация результата

            if result_image is not None:
                # Сохранение изображения с боксами
                output_image_path = f"/content/data/static/detected_{file.filename}"

                # Использование PIL для сохранения изображения
                result_image_pil = Image.fromarray(result_image)
                result_image_pil.save(output_image_path)

                # Возвращение HTML-шаблона с результатом
                return templates.TemplateResponse("image.html", {"request": request, "output_image_path": f"/static/detected_{file.filename}", "selected_class": class_select})
            else:
                # Ошибка при обработке изображения
                return templates.TemplateResponse("image.html", {"request": request, "error": "Ошибка при обработке изображения", "selected_class": class_select})
        else:
            # Объекты не найдены
            return templates.TemplateResponse("image.html", {"request": request, "error": "Объекты не найдены", "selected_class": class_select})

    except Exception as e:
        # Внутренняя ошибка сервера
        return templates.TemplateResponse("image.html", {"request": request, "error": "Internal Server Error", "selected_class": class_select})

In [ ]:
#@title HTML-шаблон для страницы распознавания изображений
image_html = """
<!DOCTYPE html>
<html>
<head>
    <!-- Подключение внешнего CSS файла и вставка стилей из переменной style -->
    <link rel="stylesheet" type="text/css" href="/static/style.css">
    <style>{style}</style>
</head>
<body>
    <!-- Верхний блок с логотипом -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo"> <!-- Изображение логотипа -->
    </div>
    <!-- Боковая панель с меню -->
    <div class="sidebar">
        <h1>Меню</h1> <!-- Заголовок меню -->
        <button class="button" onclick="window.location.href='/'">Главная</button> <!-- Кнопка "Главная" -->
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button> <!-- Кнопка "Интерфейс" -->
        <button class="button" onclick="window.location.href='/image'">Изображение</button> <!-- Кнопка "Изображение" -->
        <button class="button" onclick="window.location.href='/video'">Видео</button> <!-- Кнопка "Видео" -->
    </div>
    <!-- Основной блок контента -->
    <div class="main-content">
        <h1>Распознавание изображений</h1> <!-- Заголовок страницы -->
        <p>Загрузите изображение для распознавания объектов:</p> <!-- Инструкция для пользователя -->

        <!-- Форма для загрузки изображения и выбора класса -->
        <form action="/upload-image/" enctype="multipart/form-data" method="post">
            <input type="file" name="file" accept="image/*"> <!-- Поле для выбора файла изображения -->
            <label for="class_select">Выберите класс для распознавания:</label> <!-- Метка для выпадающего списка -->
            <select name="class_select" id="class_select"> <!-- Выпадающий список для выбора класса распознавания -->
                <option value="0" {% if selected_class == 0 %}selected{% endif %}>Человек</option> <!-- Опция для выбора "Человек" -->
                <option value="1" {% if selected_class == 1 %}selected{% endif %}>Велосипед</option> <!-- Опция для выбора "Велосипед" -->
                <option value="2" {% if selected_class == 2 %}selected{% endif %}>Машина</option> <!-- Опция для выбора "Машина" -->
            </select>
            <input type="submit" value="Загрузить" class="button"> <!-- Кнопка для загрузки файла -->
        </form>

        <!-- Отображение результата -->
        {% if output_image_path %}
            <h2>Результат:</h2> <!-- Заголовок результата -->
            <img src="{{ output_image_path }}" alt="Распознанное изображение"> <!-- Изображение с распознанными объектами -->
        {% endif %}
        {% if error %}
            <h2>{{ error }}</h2> <!-- Сообщение об ошибке, если объект не найден или произошла ошибка -->
        {% endif %}
    </div>
</body>
</html>
"""

In [ ]:
#@title Сохраняем HTML-шаблон в файл image.html
with open("/content/data/templates/image.html", "w") as f:
    f.write(image_html)

## **3. Распознование объектов на видео**

In [ ]:
#@title Функция для загрузки и обработки видео с помощью модели YOLO
@app.post("/upload-video/", response_class=HTMLResponse)
async def upload_video(request: Request, file: UploadFile = File(...), class_select: int = Form(...)):
    try:
        # Генерация уникального идентификатора для имени файла
        unique_id = str(uuid.uuid4())
        video_filename = f"{unique_id}_{file.filename}"  # Имя для загруженного видео файла
        output_filename = f"{unique_id}_detected_{file.filename}"  # Имя для обработанного видео файла
        h264_output_filename = f"{unique_id}_detected_{file.filename.replace('.mp4', '_h264.mp4')}"  # Имя для H.264 видео файла

        video_path = f"/content/data/static/{video_filename}"  # Путь для сохранения загруженного видео файла
        output_path = f"/content/data/static/{output_filename}"  # Путь для сохранения обработанного видео файла
        h264_output_path = f"/content/data/static/{h264_output_filename}"  # Путь для сохранения H.264 видео файла

        # Сохранение загруженного видео файла
        with open(video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Обработка видео с помощью модели YOLO и выбранного класса
        model = YOLO('yolov8n.pt')  # Инициализация модели YOLO
        cap = cv2.VideoCapture(video_path)  # Открытие видео файла
        if not cap.isOpened():
            raise Exception("Ошибка открытия видеофайла")

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Получение ширины видео
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Получение высоты видео
        fps = cap.get(cv2.CAP_PROP_FPS)  # Получение кадров в секунду

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для сохранения видео
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Инициализация видео писателя

        frame_count = 0  # Счетчик кадров
        object_found = False  # Флаг, найден ли объект

        while cap.isOpened():  # Обработка каждого кадра видео
            ret, frame = cap.read()
            if not ret:
                break
            results = model(frame, classes=class_select)  # Распознавание объектов на кадре
            if results[0].boxes:
                object_found = True  # Установка флага, если объект найден
            result_frame = results[0].plot()  # Визуализация результатов
            out.write(cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR))  # Сохранение кадра в видеофайл
            frame_count += 1

        cap.release()  # Освобождение захвата видео
        out.release()  # Освобождение видео писателя

        if object_found:
            # Конвертирование видео в кодек H.264 с помощью ffmpeg
            ffmpeg.input(output_path).output(h264_output_path, vcodec="libx264").run()

            # Используем прямой путь к видео
            output_video_url = f"/static/{h264_output_filename}"

            # Удаление временных файлов
            os.remove(video_path)
            os.remove(output_path)

            return templates.TemplateResponse("video.html", {"request": request, "output_video_url": output_video_url, "selected_class": class_select})
        else:
            os.remove(video_path)
            return templates.TemplateResponse("video.html", {"request": request, "error": "Объекты класса не найдены", "selected_class": class_select})

    except Exception as e:
        print(f"Ошибка: {e}")  # Логирование ошибки для отладки
        if os.path.exists(video_path):
            os.remove(video_path)
        if os.path.exists(output_path):
            os.remove(output_path)
        return templates.TemplateResponse("video.html", {"request": request, "error": "Internal Server Error", "selected_class": class_select})

In [ ]:
#@title HTML-шаблон для страницы распознавания видео
video_html = """
<!DOCTYPE html>
<html>
<head>
    <!-- Подключение внешнего CSS файла и вставка стилей из переменной style -->
    <link rel="stylesheet" type="text/css" href="/static/style.css">
    <style>{style}</style>
</head>
<body>
    <!-- Верхний блок с логотипом -->
    <div class="logo">
        <img src="/static/Logo.png" alt="Logo"> <!-- Изображение логотипа -->
    </div>
    <!-- Боковая панель с меню -->
    <div class="sidebar">
        <h1>Меню</h1> <!-- Заголовок меню -->
        <button class="button" onclick="window.location.href='/'">Главная</button> <!-- Кнопка "Главная" -->
        <button class="button" onclick="window.location.href='/interface'">Интерфейс</button> <!-- Кнопка "Интерфейс" -->
        <button class="button" onclick="window.location.href='/image'">Изображение</button> <!-- Кнопка "Изображение" -->
        <button class="button" onclick="window.location.href='/video'">Видео</button> <!-- Кнопка "Видео" -->
    </div>
    <!-- Основной блок контента -->
    <div class="main-content">
        <h1>Распознавание видео</h1> <!-- Заголовок страницы -->
        <p>Загрузите видео для распознавания объектов:</p> <!-- Инструкция для пользователя -->

        <!-- Форма для загрузки видео и выбора класса -->
        <form action="/upload-video/" enctype="multipart/form-data" method="post">
            <input type="file" name="file" accept="video/*"> <!-- Поле для выбора файла видео -->
            <label for="class_select">Выберите класс для распознавания:</label> <!-- Метка для выпадающего списка -->
            <select name="class_select" id="class_select"> <!-- Выпадающий список для выбора класса распознавания -->
                <option value="0" {% if selected_class == 0 %}selected{% endif %}>Человек</option> <!-- Опция для выбора "Человек" -->
                <option value="2" {% if selected_class == 2 %}selected{% endif %}>Машина</option> <!-- Опция для выбора "Машина" -->
                <option value="41" {% if selected_class == 41 %}selected{% endif %}>Стаканчик</option> <!-- Опция для выбора "Стаканчик" -->
            </select>
            <input type="submit" value="Загрузить" class="button"> <!-- Кнопка для загрузки файла -->
        </form>

        <!-- Отображение результата -->
        {% if output_video_url %}
            <h2>Результат:</h2> <!-- Заголовок результата -->
            <video width="800" height="600" controls> <!-- Видео плеер для отображения результата -->
                <source src="{{ output_video_url }}" type="video/mp4"> <!-- Источник видео -->
                Your browser does not support the video tag. <!-- Сообщение для старых браузеров -->
            </video>
        {% endif %}
        {% if error %}
            <h2>{{ error }}</h2> <!-- Сообщение об ошибке, если объект не найден или произошла ошибка -->
        {% endif %}
    </div>
</body>
</html>
"""

In [ ]:
#@title Сохраняем HTML-шаблон в файл video.html
with open("/content/data/templates/video.html", "w") as f:
    f.write(video_html)

## Принудительная перезагрузка

In [ ]:
#@title Принудительная перезагрузка среды и закрытие всех каналов
import os
def stop_server():
    try:
        server.should_exit = True
        server.force_exit = True  # Принудительное завершение всех задач
        server_task.cancel()
        print("Server task cancelled")
    except Exception as e:
        print(f"Error stopping server task: {e}")

    try:
        # Закрываем все активные туннели ngrok
        from pyngrok import ngrok
        for tunnel in ngrok.get_tunnels():
            ngrok.disconnect(tunnel.public_url)
        ngrok.kill()
        print("Ngrok disconnected and killed")
    except Exception as e:
        print(f"Error stopping ngrok: {e}")

    print("Server stopped")

    # Завершаем текущий сеанс
    os._exit(0)

# Вызов функции для остановки сервера
stop_server()